# Depression DBS Network Modeling in TVB

### Introduction
In this notebook we outline the project to develop a network model of the brain network directly modulated by SCCwm-DBS. This model will serve as the foundation for many projects but the immediate goal is for it to be used in improving classification accuracy in SCCwm-DBS confirmation and for likelihood ratio testing of hypothesized DBS mechanisms of action. The primary purpose of this project is to develop a data-inspired and data-congruent dynamical network model for antidepressant DBS.

Key design goals are to be able to reasonably swap between intrinsic dynamics models, swap between coupling hypothesies, switch the location of stimulation, and study the impact of typical control signals on the network's dynamics.

Forward models will also be implemented using the TVB framework to enable simulated LFP and EEG.

### Prior Work
Previous work, outlined [here](Phase Oscillator Model.ipynb), demonstrates the primary goal of this project in a TVB-independent manner. The goal is to merge in and extend this work within the framework of TVB.

### First steps

#### Preliminaries and Loading Libraries
This notebook code runs on Python 2.7. The first step is to load libraries.

In [1]:
%matplotlib notebook
import tvb
import networkx as nx
import numpy as np

from tvb.simulator.lab import *

#### Building the network model
First, we build a network model using the standard connectivity (192 parcellation). We'll use the Jansen-Rit Sigmoidal Coupling function for now.

In [2]:
KM = models.Kuramoto()

white_matter = connectivity.Connectivity.from_file("connectivity_192.zip")
white_matter.speed = np.array([4.0])
white_matter_coupling = coupling.SigmoidalJansenRit(a=10)

Next we set up the simulator with numerical details

In [3]:
sigma = np.zeros(6)
sigma[3] = 0.1

heunint = integrators.HeunStochastic(dt=2**-4,noise=noise.Additive(nsig=sigma))
mon_raw = monitors.Raw()
mon_tavg = monitors.TemporalAverage(period=2**-2)
what_to_watch = (mon_raw,mon_tavg)


sim = simulator.Simulator(model=KM,connectivity=white_matter,coupling=white_matter_coupling,integrator=heunint,monitors=what_to_watch,simulation_length=1e3)

sim.configure()
(time,data) = sim.run()

voltages = data[1]

IndexError: index 1 is out of bounds for axis 1 with size 1

#### Visualizing and forward modeling
Next, we visualize the model in the context of the 256-channel EEG recording setup. Part of this is to develop a gain model for the forward modeled EEG from the brain activity.